# Brute force extraction of $k$-local ISL transducers

In [1]:
#!/bin/python3

def prefixes(w):
    return [w[0:i] for i in range(len(w) + 1)]

def suffixes(w):
    return [w[i:] for i in range(len(w) + 1)]

def substrings(w):
    substr = [""]
    for i in range(1, len(w)+1):
        substr += ngramize(w,i)
    return substr

def ngramize(w,n):
    ngrams = []
    for i in range(len(w)-(n-1)):
        ngrams += [w[i:i+n]]
    return list(set(ngrams))

In [2]:
def create_sample(file1:str, file2:str) -> list:
    """
    Creates a .py file with the data sample in [(in, out), (in, out)]
    format.
    """

    f1 = open(file1, "r")
    f2 = open(file2, "r")

    sample = []
    for line1 in f1:
        sample.append(("_"+line1.strip()+"@", f2.readline().strip()))

    f1.close()
    f2.close()

    return sample


def alphabetize(text) -> list:
    """ Finds all symbols used in a given text. """

    alphabet = set()
    for sent in text:
        alphabet = alphabet.union(set(sent.strip()))
        if "\\" in sent:
            print(sent)
    return sorted(list(alphabet))

In [3]:
class DSFST(object):
    """ A class of Delimited Subsequential Finite State Transducers """

    def __init__(self, k:int, sigma:list):

        self.k = k
        self.sigma = sorted(sigma)
        self.states = []
        self.transitions = []


    def generate_template(self):

        # initial transition
        self.transitions.append(["", "_", [], "_"])
        self.states += ["", "_"]

        # growing the branches and connecting the last layer
        last_layer = [["", "_", [], "_"]]
        temp = []
        for round in range(self.k):
            for trans in last_layer:
                for alpha in self.sigma:
                    newstate = trans[3]+alpha
                    if len(newstate) > self.k-1:
                        newstate = newstate[-(self.k-1):]
                    if newstate not in self.states:
                        self.states.append(newstate)
                    new = [trans[3], alpha, [], newstate]
                    if new not in self.transitions:
                        self.transitions.append(new)
                    temp.append(new)
            last_layer = temp[:]
            temp = []
        del temp, last_layer

        # making everything final
        for s in self.states:
            if s != "":
                new = [s, "@", [], "@"]
                self.transitions.append(new)
        self.states.append("@")

In [4]:
def fill_options(f, S):

    # process every pair in the sample
    for pair in S:
        left = pair[0]
        right = pair[1]

        # scan the word
        prev = ""
        for i in range(len(left)):

            # read a portion of the string
            now = left[:i+1]
            if now[-1] == "@":
                now = "@"
            elif len(now) > f.k-1:
                now = now[-(f.k-1):]

            # find the transitions that corresponds to it
            current = [i for i in f.transitions if i[-1] == now and i[0] == prev][0]
            prev = now

            if current[2]:
                if current[0] == "":
                    current[2] = [i for i in current[2] if right.startswith(i)]
                elif current[-1] == "@":
                    current[2] = [i for i in current[2] if right.endswith(i)]
                else:
                    current[2] = [i for i in current[2] if i in right]
            else:
                if current[0] == "":
                    current[2] = prefixes(right)
                elif current[-1] == "@":
                    current[2] = suffixes(right)
                else:
                    current[2] = substrings(right)

In [5]:
def paths(d):
    
    print("Total layers:", len(d))
    print([len(i[2]) for i in d])
    with open('report.txt', 'w') as f:
        for i in range(len(d)):
            f.write(str(d[i])+"\n"*3)
    layers = [[i] for i in d[0][2]]
    last_layer = []
    for i in range(1, len(d)):
        print("Layer", i, "\tTotal items:", len(layers))
        for l in layers:
            for j in d[i][2][:]:
                last_layer.append(l + [j])
        layers = last_layer[:]
        last_layer = []

    return layers

In [6]:
def rewrite(s:str, fst) -> str:
    """ Reads a string and returns its rewritten version. """
    new = ""
    for i in range(len(s)):
        sofar = s[:i]
        
        if len(sofar) > fst.k-1:
            sofar = sofar[-(fst.k-1):]
        
        for j in fst.transitions:
            if j[0] == sofar and j[1] == s[i]:
                new += j[2]
                break
    
    return new

In [7]:
def gen_test_fst(S, fst):

    # rewrite unknown transition as whatever you read
    for i in fst.transitions:
        if not i[2]:
            if i[-1] == "@":
                i[2] = [""]
            else:
                i[2] = [i[1]]

    layers = paths(fst.transitions)

    print("Now constructing potential FSTs...")
    fst_list = []
    
    print("Total machines:", len(layers))
    for i in range(len(layers)):
        if str(i).endswith('00000'):
            print(i)

        current_layer = layers[i]
        new = DSFST(fst.k, fst.sigma)

        for tr in range(len(fst.transitions)):
            current_tr = fst.transitions[tr]
            new.transitions.append([current_tr[0], current_tr[1],
                                    current_layer[tr][:], current_tr[3]])

        write = True
        for data in S:
            if rewrite(data[0], new) != data[1]:
                write = False
                break
        if write:
            fst_list.append(new)

    return fst_list

## Testing

The sample maps $a \rightarrow~ b$ and $b \rightarrow~ a$.

In [8]:
S = [["aba", "bab"], ["bab", "aba"], ["a", "b"], ["b", "a"], ["ab", "ba"], ["ba", "ab"], ["baa", "abb"]]
for i in range(len(S)):
    S[i][0] = "_"+S[i][0]+"@"

In [9]:
import time
start_time = time.time()

# parameters for the FST
k = 2
sigma = ["a", "b"]

# generates the FST template
f = DSFST(k, sigma)
f.generate_template()
print("FST template generated.")

# filling all the options in the FST (VERY SLOW)
fill_options(f, S)
print("Template with at least one successful run generated.")

# generate FSTs
good_list = gen_test_fst(S, f)
print("Generated appropriate FSTs,", len(good_list), "total.")

end_time = time.time()

print("Execution time:", end_time - start_time)

FST template generated.
Template with at least one successful run generated.
Total layers: 10
[1, 2, 2, 6, 4, 4, 1, 1, 2, 2]
Layer 1 	Total items: 1
Layer 2 	Total items: 2
Layer 3 	Total items: 4
Layer 4 	Total items: 24
Layer 5 	Total items: 96
Layer 6 	Total items: 384
Layer 7 	Total items: 384
Layer 8 	Total items: 384
Layer 9 	Total items: 768
Now constructing potential FSTs...
Total machines: 1536
Generated appropriate FSTs, 4 total.
Execution time: 0.020495176315307617


In [10]:
test = ["_aba@", "_baaa@", "_aaaa@", "_bbabb@"]
number = 1
for i in good_list:
    print("Transducer", number)
    for string in test:
        print(string[1:-1], "\t->\t", rewrite(string, i))
    number += 1
    print("\n")

Transducer 1
aba 	->	 bab
baaa 	->	 abbb
aaaa 	->	 bbbb
bbabb 	->	 babba


Transducer 2
aba 	->	 bab
baaa 	->	 abbb
aaaa 	->	 bbbb
bbabb 	->	 abbab


Transducer 3
aba 	->	 bab
baaa 	->	 abbb
aaaa 	->	 bbbb
bbabb 	->	 babba


Transducer 4
aba 	->	 bab
baaa 	->	 abbb
aaaa 	->	 bbbb
bbabb 	->	 abbab




In [11]:
for i in good_list:
    print(i.transitions)
    print("\n")

[['', '_', '', '_'], ['_', 'a', '', 'a'], ['_', 'b', '', 'b'], ['a', 'a', 'b', 'a'], ['a', 'b', 'b', 'b'], ['b', 'a', 'a', 'a'], ['b', 'b', 'b', 'b'], ['_', '@', '', '@'], ['a', '@', 'b', '@'], ['b', '@', 'a', '@']]


[['', '_', '', '_'], ['_', 'a', '', 'a'], ['_', 'b', 'a', 'b'], ['a', 'a', 'b', 'a'], ['a', 'b', 'ba', 'b'], ['b', 'a', '', 'a'], ['b', 'b', 'b', 'b'], ['_', '@', '', '@'], ['a', '@', 'b', '@'], ['b', '@', '', '@']]


[['', '_', '', '_'], ['_', 'a', 'b', 'a'], ['_', 'b', '', 'b'], ['a', 'a', 'b', 'a'], ['a', 'b', '', 'b'], ['b', 'a', 'ab', 'a'], ['b', 'b', 'b', 'b'], ['_', '@', '', '@'], ['a', '@', '', '@'], ['b', '@', 'a', '@']]


[['', '_', '', '_'], ['_', 'a', 'b', 'a'], ['_', 'b', 'a', 'b'], ['a', 'a', 'b', 'a'], ['a', 'b', 'a', 'b'], ['b', 'a', 'b', 'a'], ['b', 'b', 'b', 'b'], ['_', '@', '', '@'], ['a', '@', '', '@'], ['b', '@', '', '@']]


